In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random


model_name = 'eryk-mazus/polka-1.1b'
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [3]:
vocab = tokenizer.get_vocab()

In [4]:
vocab["▁Nau"]

26700

In [15]:
import re

def split_string_by_punctuation_and_whitespace(s: str) -> list:
    return re.split(r'[\s\W]+', s)

def restricted_sampling(input_ids):
	with torch.no_grad():
		outputs = model(torch.Tensor([input_ids]).long().to(device))
		next_token_logits = outputs.logits[:, -1, :]  # Get logits for the last token

	probs = F.softmax(next_token_logits, dim=-1).squeeze()
	for token_id in range(len(probs)):
		if not tokenizer.convert_ids_to_tokens(token_id):
			probs[token_id] = 0
			continue

		sentence = tokenizer.decode(input_ids + [token_id], skip_special_tokens=True)
		split = split_string_by_punctuation_and_whitespace(sentence)
		prefix = sentence[0]
		if not all(string.startswith(prefix) or not string for string in split):
			probs[token_id] = 0

	probs /= probs.sum()
	return torch.multinomial(probs, num_samples=1).item()

def extend_sentence(sentence_txt, iters=30):
	sentence_txt = sentence_txt.lower()
	letter = sentence_txt[0]
	token_chain = list(tokenizer.encode(sentence_txt))

	for i in range(iters):
		token_chain.append(restricted_sampling(token_chain))
		print(tokenizer.decode(token_chain, skip_special_tokens=True))
	return token_chain

In [17]:
sentence = extend_sentence("Nowa nawierzchnia na")

nowa nawierzchnia na n
nowa nawierzchnia na naram
nowa nawierzchnia na naramow
nowa nawierzchnia na naramowick
nowa nawierzchnia na naramowickiej
nowa nawierzchnia na naramowickiej.
nowa nawierzchnia na naramowickiej. -
nowa nawierzchnia na naramowickiej. - nab
nowa nawierzchnia na naramowickiej. - nabór
nowa nawierzchnia na naramowickiej. - nabórciąż
nowa nawierzchnia na naramowickiej. - nabórciążone
nowa nawierzchnia na naramowickiej. - nabórciążone -
nowa nawierzchnia na naramowickiej. - nabórciążone - n
nowa nawierzchnia na naramowickiej. - nabórciążone - nied
nowa nawierzchnia na naramowickiej. - nabórciążone - niedaw
nowa nawierzchnia na naramowickiej. - nabórciążone - niedawno
nowa nawierzchnia na naramowickiej. - nabórciążone - niedawno -
nowa nawierzchnia na naramowickiej. - nabórciążone - niedawno - n
nowa nawierzchnia na naramowickiej. - nabórciążone - niedawno - nisk
nowa nawierzchnia na naramowickiej. - nabórciążone - niedawno - niskie
nowa nawierzchnia na naramowickiej. -